In [157]:
import os
import pandas as pd
import numpy as np

import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [158]:
path = os.path.join(os.path.normpath(os.getcwd() + os.sep + os.pardir) + "/data/raw/")

In [159]:
os.listdir(path)

['BX-NewBooksUsers.csv',
 'BX-Books.csv',
 'BX-Ratings.csv',
 'BX-Users.csv',
 'BX-NewBooksRatings.csv',
 'BX-NewBooks.csv']

In [312]:
users = pd.read_csv(path + 'BX-Users.csv')
ratings = pd.read_csv(path + 'BX-Ratings.csv')
books = pd.read_csv(path + 'BX-Books.csv')

In [313]:
ratings_user = ratings.merge(users)
ratings_user = ratings_user[['User-ID', 'ISBN', 'Book-Rating', 'User-Age']]

In [314]:
ratings_user

,User-ID,ISBN,Book-Rating,User-Age
0,276744,038550120X,7,NaN
1,276754,0684867621,8,19
2,276755,0451166892,5,32
3,276762,0380711524,5,25
4,276772,0553572369,7,NaN
...,...,...,...,...
204159,276688,0892966548,10,NaN
204160,276688,1551669315,6,NaN
204161,276704,0345386108,6,NaN
204162,276704,0743211383,7,NaN


In [315]:
books_read = ratings_user.groupby(['User-ID'])["ISBN"].count().reset_index()
books_read[books_read['ISBN'] > 25]

,User-ID,ISBN
40,254,37
81,507,28
97,638,45
148,882,30
252,1424,34
...,...,...
47894,276680,64
48014,277427,87
48143,278026,36
48215,278418,37


In [316]:
ratings_user = ratings_user[ratings_user['User-ID'].isin(books_read[books_read['ISBN'] > 20]['User-ID'])]

In [317]:
ratings_user = pd.pivot(ratings_user, index = ['User-ID', 'User-Age'], values = 'Book-Rating', columns = 'ISBN').reset_index()

In [270]:
from sklearn.decomposition import NMF

In [273]:
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(features)

In [274]:
H = model.components_

In [287]:
features = pd.DataFrame(np.dot(W, H))

In [288]:
features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,17054,17055,17056,17057,17058,17059,17060,17061,17062,17063,17064,17065,17066,17067,17068,17069,17070,17071,17072,17073,17074,17075,17076,17077,17078,17079,17080,17081,17082,17083,17084,17085,17086,17087,17088,17089,17090,17091,17092,17093
0,8.188517,8.188374,8.187222,8.191095,8.187320,8.189671,8.187311,8.189552,8.191717,8.190032,8.190555,8.190262,8.189515,8.189946,8.189506,8.186530,8.189843,8.188435,8.189605,8.190123,8.191326,8.189825,8.188520,8.188734,8.188277,8.190378,8.187085,8.189005,8.189228,8.187834,8.189962,8.187892,8.190303,8.187982,8.188671,8.185194,8.183064,8.187088,8.187696,8.186478,...,8.189029,8.189323,8.189666,8.189817,8.187561,8.187909,8.188689,8.189003,8.187649,8.188769,8.186106,8.189531,8.188416,8.189014,8.189945,8.188966,8.188953,8.190750,8.190139,8.189596,8.190177,8.188999,8.190308,8.190576,8.187751,8.188974,8.188902,8.187490,8.190987,8.195661,8.191600,8.192315,8.189029,8.190444,8.190210,8.187237,8.189975,8.185931,8.189368,8.189862
1,8.141325,8.141325,8.138596,8.144160,8.139534,8.144296,8.139431,8.141109,8.144027,8.143151,8.145649,8.142888,8.143583,8.142025,8.141003,8.142910,8.143294,8.142329,8.141058,8.143681,8.146302,8.140883,8.144388,8.145157,8.144172,8.144785,8.140948,8.140883,8.143945,8.140599,8.141295,8.140588,8.144511,8.139788,8.141796,8.137175,8.137039,8.143417,8.141334,8.139014,...,8.142312,8.142310,8.141569,8.141958,8.140303,8.143663,8.141361,8.140825,8.140228,8.140360,8.138180,8.140621,8.141398,8.141959,8.141832,8.142692,8.140527,8.142301,8.141413,8.142156,8.142326,8.141608,8.145767,8.146107,8.139233,8.140083,8.140917,8.139924,8.147238,8.148165,8.145449,8.143757,8.141166,8.144373,8.143479,8.140009,8.141658,8.140789,8.141521,8.142233
2,9.354994,9.354749,9.354360,9.357781,9.353980,9.355246,9.354023,9.356906,9.358932,9.356539,9.355979,9.357090,9.355394,9.357048,9.356889,9.350641,9.356128,9.354265,9.357027,9.356386,9.356926,9.357509,9.353207,9.353126,9.352914,9.356178,9.352746,9.356095,9.354687,9.354242,9.357505,9.354349,9.356210,9.354972,9.354985,9.351694,9.348101,9.351305,9.353575,9.352832,...,9.355299,9.355809,9.356834,9.356867,9.353945,9.352577,9.355271,9.356127,9.354141,9.355994,9.352679,9.357156,9.354778,9.355481,9.357161,9.354969,9.356215,9.358274,9.357740,9.356370,9.357272,9.355660,9.355483,9.355746,9.354898,9.356510,9.355898,9.354045,9.355794,9.363311,9.357898,9.360121,9.355971,9.356533,9.356654,9.353557,9.357314,9.350850,9.356348,9.356782
3,8.498990,8.498845,8.497611,8.501672,8.497733,8.500233,8.497721,8.500033,8.502300,8.500571,8.501162,8.500797,8.500056,8.500455,8.499984,8.497015,8.500382,8.498931,8.500086,8.500676,8.501959,8.500305,8.499068,8.499304,8.498816,8.500961,8.497529,8.499474,8.499775,8.498280,8.500454,8.498339,8.500878,8.498410,8.499157,8.495520,8.493358,8.497593,8.498159,8.496867,...,8.499533,8.499831,8.500161,8.500323,8.497996,8.498431,8.499165,8.499471,8.498084,8.499221,8.496470,8.500000,8.498890,8.499509,8.500450,8.499479,8.499412,8.501277,8.500636,8.500104,8.500697,8.499485,8.500914,8.501194,8.498162,8.499423,8.499370,8.497915,8.501639,8.506399,8.502216,8.502899,8.499505,8.501017,8.500759,8.497660,8.500475,8.496356,8.499858,8.500375
4,7.085702,7.085665,7.083707,7.088098,7.084302,7.087808,7.084237,7.085842,7.088181,7.087207,7.088860,7.087108,7.087334,7.086502,7.085766,7.086144,7.087245,7.086290,7.085825,7.087553,7.089458,7.085777,7.087562,7.088084,7.087367,7.088289,7.085098,7.085562,7.087479,7.085083,7.086062,7.085091,7.088104,7.084629,7.086029,7.082313,7.081673,7.086598,7.085493,7.083765,...,7.086436,7.086511,7.086151,7.086427,7.084832,7.086962,7.085769,7.085526,7.084809,7.085183,7.083162,7.085541,7.085720,7.086217,7.086384,7.086650,7.085332,7.086880,7.086181,7.086489,7.086745,7.086000,7.088867,7.089143,7.084232,7.085069,7.085556,7.084583,7.089938,7.091725,7.089013,7.088173,7.085740,7.088057,7.087453,7.084568,7.086286,7.084700,7.086044,7.086606
...,...,...,...

In [279]:
ratings_user["User-Age"] = ratings_user["User-Age"].str.rstrip('"')
ratings_user['User-Age'] = ratings_user['User-Age'].astype('Int64')

AttributeError: Can only use .str accessor with string values!

In [289]:
features = ratings_user[ratings_user.columns[2:]]
target = ratings_user['User-Age']

In [290]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [291]:
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(features)

NearestNeighbors(metric='euclidean')

In [292]:
def impute_age(row):
    if pd.isna(row['User-Age']):
        # Find indices of k-nearest neighbors
        distances, indices = knn.kneighbors([features.iloc[row.name]])
        nearest_neighbors_ages = target.iloc[indices.flatten()].dropna()
        if not nearest_neighbors_ages.empty:
            imputed_age = nearest_neighbors_ages.median()
            return imputed_age
    return row['User-Age']

In [293]:
ratings_user['User-Age-Imputed'] = ratings_user.apply(impute_age, axis=1)

/Users/caseyhaseloff/anaconda3/lib/python3.11/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/caseyhaseloff/anaconda3/lib/python3.11/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/caseyhaseloff/anaconda3/lib/python3.11/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/caseyhaseloff/anaconda3/lib/python3.11/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/caseyhaseloff/anaconda3/lib/python3.11/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/User

In [294]:
user_age = ratings_user[['User-ID', 'User-Age', 'User-Age-Imputed']]

In [295]:
l = list(user_age.iloc[knn.kneighbors([features.iloc[1]])[1][0]]['User-ID'])
l.append(user_age.iloc[1]['User-ID'])
l

/Users/caseyhaseloff/anaconda3/lib/python3.11/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


[507.0, 18082, 213523.0, 270554.0, 238613, 507.0]

In [296]:
books_read[books_read['User-ID'].isin(l)]

,User-ID,ISBN
81,507,28
2930,18082,31
36895,213523,24
41257,238613,31
46935,270554,31


In [298]:
users[users['User-ID'].isin(l)]

,User-ID,User-City,User-State,User-Country,User-Age
81,507,dumas,arkansas,"usa""",NaN
2930,18082,southern,california,"usa""",38
36895,213523,brookville,pennsylvania,"usa""",NaN
41257,238613,st. john`s,newfoundland,"canada""",37
46935,270554,fredericton,new brunswick,"canada""",NaN


In [297]:
user_age[user_age['User-Age'].isna()]['User-Age-Imputed'].unique()

array([ 37.5,  25. ,  38. ,  20. ,  39. ,  48. ,  38.5,  30. ,  33.5,
        44. ,  28. ,  55. ,  35. ,  25.5,  31. ,  49. ,  51. ,  49.5,
        54.5,  52. ,  33. ,  28.5,  37. ,  21. ,  42.5,  34. ,  29. ,
        48.5, 101. ,  43. ,  50.5,  36. ,  23.5,  32. ,  29.5,  40. ,
        42. ,  39.5,  36.5,  53. ,  27. ,  22. ,  55.5,  26. ,  22.5,
        47. ,  50. ,  66.5,  44.5,  23. ,  41.5,  19. ,  30.5,  35.5,
        21.5,  41. ,  45. ,  46.5,  18. ,  27.5,  67. ,  26.5,  24.5,
        24. ,  45.5])

In [299]:
book_ratings = ratings.merge(books)

In [300]:
book_ratings = book_ratings[book_ratings['User-ID'].isin(l)].sort_values('Book-Author')

In [301]:
book_ratings

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Book-Publisher
199990,270554,0375412824,8,The Dive From Clausen's Pier (Alex Awards),ANN PACKER,2002,Knopf
176561,238613,0345313860,9,"The Vampire Lestat (Vampire Chronicles, Book II)",ANNE RICE,1986,Ballantine Books
16787,18082,0425117383,8,At Risk,Alice Hoffman,1989,Berkley Publishing Group
199984,270554,0316096199,9,Lucky : A Memoir,Alice Sebold,2002,Back Bay Books
200001,270554,0804114986,8,The Bonesetter's Daughter,Amy Tan,2002,Ballantine Books
...,...,...,...,...,...,...,...
16800,18082,0515136565,8,Final Justice: A Badge of Honor (Badge of Honor),W. E. B. Griffin,2003,Jove Books
199975,270554,0060987561,10,I Know This Much Is True,Wally Lamb,1999,Regan Books
199999,270554,0671003755,8,She's Come Undone (Oprah's Book Club (Paperback)),Wally Lamb,1996,Washington Square Press
199992,270554,0385510012,7,The Swallows of Kabul : A Novel,YASMINA KHADRA,2004,Nan A. Talese


In [302]:
book_ratings[book_ratings['Book-Title'].duplicated()]

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Book-Publisher
1446,507,0767912918,7,Passing for Thin : Losing Half My Weight and F...,FRANCES KUFFEL,2004,Broadway
158012,213523,0553576801,9,The Perfect Husband,LISA GARDNER,1997,Bantam
176572,238613,0446605239,8,The Notebook,Nicholas Sparks,1998,Warner Books
176559,238613,0142001740,9,The Secret Life of Bees,Sue Monk Kidd,2003,Penguin Books


In [307]:
book_ratings[book_ratings['ISBN'] == '0767912918']

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Book-Publisher
16807,18082,0767912918,9,Passing for Thin : Losing Half My Weight and F...,FRANCES KUFFEL,2004,Broadway
1446,507,0767912918,7,Passing for Thin : Losing Half My Weight and F...,FRANCES KUFFEL,2004,Broadway


In [308]:
book_ratings[book_ratings['Book-Title'] == 'The King of Torts']

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Book-Publisher


In [323]:
a = ratings_user[ratings_user['User-ID'].isin(l)].T.reset_index().merge(books)

In [329]:
a[a['ISBN'].isin(book_ratings['ISBN'])].sort_values('Book-Author').tail(50)

,ISBN,1,92,1249,1398,1595,Book-Title,Book-Author,Year-Of-Publication,Book-Publisher
7811,0446608955,NaN,NaN,NaN,NaN,6.0,A Walk to Remember,Nicholas Sparks,2000,Warner Books
10042,0553272837,NaN,NaN,8.0,NaN,NaN,Brazen Virtue,Nora Roberts,1996,Bantam Books
9995,0553265741,NaN,NaN,9.0,NaN,NaN,Sacred Sins,Nora Roberts,1990,Bantam Books
14488,0812533534,10.0,NaN,NaN,NaN,NaN,Seventh Son (Tales of Alvin Maker (Paperback)),Orson Scott Card,1988,Tor Books
12649,0679893105,9.0,NaN,NaN,NaN,NaN,"The Golden Compass (His Dark Materials, Book 1)",PHILIP PULLMAN,1998,Knopf Books for Young Readers
10030,0553271369,NaN,8.0,NaN,NaN,NaN,The Lords of Discipline,Pat Conroy,1986,Bantam Books
13243,0743227441,NaN,NaN,NaN,9.0,NaN,The Other Boleyn Girl,Philippa Gregory,2002,Touchstone
10140,0553285246,NaN,NaN,NaN,8.0,NaN,Prince of the Blood,RAYMOND FEIST,1990,Spectra
9982,0553263285,NaN,NaN,NaN,8.0,NaN,A Darkness at Sethanon : Volume IV in the Rift...,RAYMOND FEIST,1987,Spectra
14155,080410526X,NaN,9.0,NaN,NaN,NaN,All I Really Need to Know,ROBERT FULGHUM,1989,Ivy Books


In [242]:
a[1]['0002005018']

8.142857142857142

In [262]:
book_ratings

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Book-Publisher
137232,182838,0553579002,8,"Solo Command (Star Wars: X-Wing Series, Book 7)",Aaron Allston,1999,Bantam
61737,81121,0060830956,9,Brave New World,Aldous Huxley,1984,HarperCollins Publishers
16787,18082,0425117383,8,At Risk,Alice Hoffman,1989,Berkley Publishing Group
137210,182838,0345409329,8,"Blood and Gold (Rice, Anne, Vampire Chronicles.)",Anne Rice,2002,Ballantine Books
16792,18082,0446604275,8,Primary Colors: A Novel of Politics,Anonymous,1996,Warner Books
...,...,...,...,...,...,...,...
61746,81121,0441117732,8,Count Zero (Ace Science Fiction),William Gibson,1987,Ace Books
61749,81121,0441569595,10,Neuromancer (Remembering Tomorrow),William Gibson,1995,Ace Books
61751,81121,0553281747,8,Mona Lisa Overdrive (Bantam Spectra Book),William Gibson,1989,Bantam
61758,81121,0812694333,9,The Simpsons and Philosophy: The D'oh! of Homer,William Irwin,2001,Open Court Publishing Company
